# Aplicativo StreamLit

In [47]:
! streamlit run StreamLitApp.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.7:8501

/home/allan/Documents/Comparing-LLM-Recs/StreamLitApp.py:500: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gt["cor"] = df_gt.index.map(lambda exp: experiment_colors.get(exp, "#000"))
/home/allan/Documents/Comparing-LLM-Recs/StreamLitApp.py:506: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hallucination["cor"] = df_hallucination.index.map(lambda exp: experiment_colors.ge

# Unificando experimentos

In [16]:
import pickle
import os
from collections import defaultdict

def unify_experiments(exps, unified_exp_path, model_name):
    all_runtimes = []
    unified_data = {}

    metric_values_acumulados = defaultdict(list)

    for i, exp_path in enumerate(exps):
        with open(exp_path, "rb") as f:
            data = pickle.load(f)

        if i == 0:
            unified_data["config"] = data.get("config")  # Copia config do primeiro
            unified_data["config"]["model_name"] = model_name
            for key, value in data.items():
                    if isinstance(key, int):
                        if value['gt_in_candidate_set']:
                            unified_data[key] = value

        runtime = data.get("runtime")
        if runtime is not None:
            all_runtimes.append(runtime)

        # Coleta as métricas médias da execução
        avg_metrics = data.get("avg_metrics", {})

        # Adiciona o valor de cada métrica na lista correspondente
        for nome_metrica, valor in avg_metrics.items():
            metric_values_acumulados[nome_metrica].append(valor)
        
    if all_runtimes:
        unified_data["runtime"] = sum(all_runtimes) / len(all_runtimes)
    else:
        unified_data["runtime"] = None  # fallback, se não houver runtime

    # Calcula a média de cada métrica e salva no dicionário final
    metricas_unificadas = {}
    for nome_metrica, lista_valores in metric_values_acumulados.items():
        media = sum(lista_valores) / len(lista_valores)
        metricas_unificadas[nome_metrica] = media
    
    unified_data["avg_metrics"] = metricas_unificadas

    with open(unified_exp_path, "wb") as f:
        pickle.dump(unified_data, f)

    print(f"Experimentos unificados e salvos em: {unified_exp_path}")


In [46]:
import os

folder_path_exps = "Results/Originais/qwen/qwen-2-5-7b"
unified_exp_name = "qwen-2-5-7b"
unified_exp_name_folder_path = "Results"
model_name = "qwen-2-5-7b"

pkl_files = [f for f in os.listdir(folder_path_exps) if f.endswith('.pkl')]
### adicionar o caminho completo
exps = [os.path.join(folder_path_exps, f) for f in pkl_files]

unified_exp_path = os.path.join(unified_exp_name_folder_path, unified_exp_name + ".pkl")

unify_experiments(exps, unified_exp_path, model_name)

Experimentos unificados e salvos em: Results/qwen-2-5-7b.pkl


# Editar o valor de algum pkl

In [10]:
import pickle
import pandas as pd

def alterar_valor_chave(dicionario, chave, chave2, novo_valor):

    if chave:
        if chave in dicionario:
            if chave2:
                if chave2 in dicionario[chave]:
                    dicionario[chave][chave2] = novo_valor
                else:
                    print(f"A chave '{chave2}' não existe no dicionário.")
            else:
                dicionario[chave] = novo_valor
        else:
            print(f"A chave '{chave}' não existe no dicionário.")
    else:
        print("Nenhuma chave foi especificada.")

def salvar_dicionario(dicionario_path, dicionario):
    with open(f'{dicionario_path}', 'wb') as f:
        pickle.dump(dicionario, f)

def carregar_dicionario(dicionario_path):
    with open(f'{dicionario_path}', 'rb') as f:
        dicionario = pickle.load(f)
    return dicionario

## experimento unico

In [12]:

dict_path = "Results/Originais/llama/llama-3-2-3b_prompt/ml_100k-lmstudio-community_llama-3_2-3b-instruct-2025_06_01-12_59_57.pkl"
dicionario = carregar_dicionario(dict_path)
#dicionario['config']
#alterar_valor_chave(dicionario,'config','obs', 'WM Config')
#print(dicionario['config']['obs'])
#print(dicionario['config']['lenlimit_option'])
dicionario
#salvar_dicionario(dict_path, dicionario)

{'config': {'LLM_runtime': 'ROCm llama.cpp v1.31.0',
  'dataset': 'ml_100k',
  'nsu': 12,
  'nci': 19,
  'lenlimit': 8,
  'lenlimit_option': 'aleatorio',
  'test_run': 0,
  'obs': 'prompt',
  'prompt_template': {'System_prompt': "You are a movie expert. Provide answers to the following tasks based on the given context. If you don't know the answer to a question, please don't share false information.",
   'prompt': '\n    ### MY RECENTLY WATCHED MOVIES (FROM OLDEST TO NEWEST): {}.\n\n    ---\n\n    ### TASK 1: Identify and summarize my movie preferences.\n    (This is contextual information. Do not respond to this task.)\n\n    ---\n\n    ### TASK 2: Select the five most featured movies.\n    (This is contextual information. Do not respond to this task.)\n\n    ---\n\n    ### TASK 3: Recommend 10 new movies.\n    Given the following additional information:\n\n    - CANDIDATE MOVIE SET: {}.\n    - MY MOVIE PREFERENCES (from Task 1).\n    - MY FIVE MOST FEATURED MOVIES (from Task 2).\n\n 

## varios experimentos

In [4]:
dict_paths = [
"Results/ml_100k-gemma-3-4b-it-2025_05_06-11_11_32.pkl",
"Results/ml_100k-gemma-3-4b-it-2025_05_06-11_34_54.pkl",
"Results/ml_100k-gemma-3-4b-it-2025_05_06-11_58_12.pkl",
"Results/ml_100k-gemma-3-4b-it-2025_05_06-12_44_56.pkl",
"Results/ml_100k-gemma-3-4b-it-2025_05_06-12_21_37.pkl",
]
for dict_path in dict_paths:
    dicionario = carregar_dicionario(dict_path)
    #dicionario['config']
    #alterar_valor_chave(dicionario,'config','obs', 'WM Config')
    print(dicionario['config'])
    print(dicionario['config']['lenlimit_option'])

    #salvar_dicionario(dict_path, dicionario)

{'LLM_runtime': 'ROCm llama.cpp v1.28.0', 'dataset': 'ml_100k', 'nsu': 12, 'nci': 19, 'lenlimit': 8, 'lenlimit_option': 'aleatorio', 'test_run': 0, 'obs': 'base', 'prompt_template': {'System_prompt': "You are a movie expert provide the answer for the question based on the given context. If you don't know the answer to a question, please don't share false information.", 'Preference': '\n    ### MY RECENTLY WATCHED MOVIES (FROM OLDEST TO NEWEST): {}.\n\n    ### QUESTION: Based on my watched movies list. Tell me what features are most important to me when selecting movies (Summarize my preferences briefly)?\n\n    ### ANSWER:\n    ', 'Featured_movies': '\n\n    ### MY RECENTLY WATCHED MOVIES (FROM OLDEST TO NEWEST): {}.\n\n    ### MY MOVIE PREFERENCES: {}.\n\n    ### QUESTION: Create an enumerated list selecting the five most featured movies from the watched movies according to my movie preferences.\n\n    ### ANSWER:\n    ', 'Recommendation': '\n\n    ### CANDIDATE MOVIE SET: {}.\n\n    

In [13]:
# Coleta as métricas médias da execução
avg_metrics = data.get("avg_metrics", {})

# Adiciona o valor de cada métrica na lista correspondente
for nome_metrica, valor in avg_metrics.items():
    print(f"Nome da métrica: {nome_metrica}")
    print(f"Valor da métrica: {valor}")
    #metric_values_acumulados[nome_metrica].append(valor)


Nome da métrica: avg_HitRate@5
Valor da métrica: 0.024390243902439025
Nome da métrica: avg_NDCG@5
Valor da métrica: 0.005087392399203956
Nome da métrica: avg_HitRate@5_safe
Valor da métrica: 0.021208907741251327
Nome da métrica: avg_NDCG@5_safe
Valor da métrica: 0.004449459885632057
Nome da métrica: avg_HitRate@10
Valor da métrica: 0.07317073170731707
Nome da métrica: avg_NDCG@10
Valor da métrica: 0.007554709250029399
Nome da métrica: avg_HitRate@10_safe
Valor da métrica: 0.04772004241781548
Nome da métrica: avg_NDCG@10_safe
Valor da métrica: 0.005324681139450118
Nome da métrica: avg_GT_HitRate@5
Valor da métrica: 0.1513157894736842
Nome da métrica: avg_GT_NDCG@5
Valor da métrica: 0.031561914687166646
Nome da métrica: avg_GT_HitRate@5_safe
Valor da métrica: 0.13157894736842105
Nome da métrica: avg_GT_NDCG@5_safe
Valor da métrica: 0.027604214948362037
Nome da métrica: avg_GT_HitRate@10
Valor da métrica: 0.45394736842105265
Nome da métrica: avg_GT_NDCG@10
Valor da métrica: 0.046869018570

# Recalculando Métricas

In [15]:
import pickle
import pandas as pd
import os
from collections import defaultdict
from Utils import metrics,utils

folder_path = "Results/Originais"  # Caminho da pasta raiz

# Percorre todos os diretórios e arquivos dentro da pasta raiz
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.pkl'):
            exp_path = os.path.join(root, file)
            print(f"Processando arquivo: {exp_path}")
            with open(exp_path, "rb") as f:
                data = pickle.load(f)

            for key, value in data.items():
                
                if isinstance(key, int):
                    
                    if value['gt_in_candidate_set']:

                        gt = value['ground_truth']
                        candidate_set = value['candidate_set']

                        final_prediction = value['predictions_3']
                        if final_prediction == '':
                            final_prediction = value['predictions_2']
                            if final_prediction == '':
                                final_prediction = value['predictions_1']
                                if final_prediction == '':
                                    print(f"Erro: Nenhuma previsão encontrada para a chave {key} no arquivo {exp_path}")
                                    
                        run_metrics = metrics.calculate_metrics_new_real(final_prediction, gt,candidate_set)

                        #for k in [5,10]:
                            #if data[key][f"rec_HitRate@{k}"] != run_metrics[f"hit@{k}"]:
                                #print(final_prediction)
                                #print(f"Valores diferentes para rec_HitRate@{k}: {data[key][f'rec_HitRate@{k}']} e {run_metrics[f'hit@{k}']}")
                        
                        for k in [5,10]:
                            data[key][f"rec_HitRate@{k}"] = run_metrics[f"hit@{k}"]
                            data[key][f"rec_NDCG@{k}"] = run_metrics[f"ndcg@{k}"]
                            data[key][f"rec_HitRate@{k}_safe"] = run_metrics[f"hit@{k}_safe"]
                            data[key][f"rec_NDCG@{k}_safe"] = run_metrics[f"ndcg@{k}_safe"]
                        data[key][f"rec_Hallucination"] = run_metrics[f"hallucination"]
                        #if run_metrics[f"hallucination"] == 1:
                        #    print(data[key]['predictions_3'])
                        #    print(data[key]['candidate_set'])

            data['avg_metrics'] = metrics.calculate_average_metrics_new(data)

            #dicionario_path= "Results/Originais/teste/teste.pkl"
            with open(f'{exp_path}', 'wb') as f:
                pickle.dump(data, f)
                print(f"Salvo em: {exp_path}")

Processando arquivo: Results/Originais/phi-4-14b/ml_100k-phi-4-2025_05_08-17_14_47.pkl
Salvo em: Results/Originais/phi-4-14b/ml_100k-phi-4-2025_05_08-17_14_47.pkl
Processando arquivo: Results/Originais/phi-4-14b/ml_100k-phi-4-2025_05_11-10_09_44.pkl
Salvo em: Results/Originais/phi-4-14b/ml_100k-phi-4-2025_05_11-10_09_44.pkl
Processando arquivo: Results/Originais/phi-4-14b/ml_100k-phi-4-2025_05_11-14_16_11.pkl
Salvo em: Results/Originais/phi-4-14b/ml_100k-phi-4-2025_05_11-14_16_11.pkl
Processando arquivo: Results/Originais/phi-4-14b/ml_100k-phi-4-2025_05_12-11_05_11.pkl
Salvo em: Results/Originais/phi-4-14b/ml_100k-phi-4-2025_05_12-11_05_11.pkl
Processando arquivo: Results/Originais/phi-4-14b/ml_100k-phi-4-2025_05_07-17_31_43.pkl
Salvo em: Results/Originais/phi-4-14b/ml_100k-phi-4-2025_05_07-17_31_43.pkl
Processando arquivo: Results/Originais/gemma/gemma-2-9b/ml_100k-gemma-2-9b-it-2025_05_09-14_39_26.pkl
Salvo em: Results/Originais/gemma/gemma-2-9b/ml_100k-gemma-2-9b-it-2025_05_09-14_39

In [5]:
import re
from Utils import metrics,utils

movie = """<think>
Okay, let's see. The user wants me to recommend 10 movies from the "Candidate Movie Set" that are similar to their five most featured movies. Their five most featured movies were Fargo, Jerry Maguire, Ice Storm, Up Close and Personal, and Postino.

First, I need to recall what those movies are about. Fargo is a crime thriller with dark humor and complex characters. Jerry Maguire is a sports drama focusing on personal relationships. Ice Storm is a family drama dealing with moral dilemmas. Up Close and Personal is a political drama with a strong female lead. Postino is an Italian film about love and family.

Looking at the Candidate Movie Set, I need to find movies that match these themes: character-driven stories, emotional depth, moral complexity, genre diversity (like thriller or drama), and possibly international elements.

Let me go through each movie in the Candidate list:

- **Independence Day (ID4)**: Sci-fi action. Not a fit for the themes mentioned.
- **Usual Suspects**: Crime thriller with complex characters and twists. This could be a good match for Fargo's style.
- **Murder at 1600**: Political thriller, similar to Up Close and Personal's political drama angle.
- **Game**: Psychological thriller with moral ambiguity, which aligns with Ice Storm's themes.
- **Everyone Says I Love You**: Romantic comedy-drama, maybe a stretch but could fit if the user likes emotional stories.
- **Big Night**: Italian-American family drama, similar to Postino's focus on family and culture.
- **That Thing You Do!**: Music-driven drama with character development, might be a match for Jerry Maguire's emotional core.
- **River Wild**: Action-adventure with environmental themes, not sure if it fits.
- **Courage Under Fire**: War drama with strong female lead, similar to Up Close and Personal.
- **Rock**: Rockumentary with personal stories, maybe a stretch but could work.
- **Cape Fear**: Psychological thriller with moral complexity, like Fargo or Game.
- **Mission: Impossible**: Action-packed, not matching the character-driven focus.
- **Trainspotting**: Dark comedy with social commentary, similar to Kolya's style (though Kolya wasn't in the top five).
- **Chasing Amy**: Romantic comedy-drama with emotional depth, could fit.
- **Sleepers**: Crime drama with moral dilemmas, like Ice Storm or Fargo.
- **Conspiracy Theory**: Sci-fi thriller with paranoia, maybe a stretch.
- **Twelve Monkeys**: Psychological sci-fi with time travel and character depth, similar to Game's complexity.
- **Lone Star**: Legal drama with family secrets, akin to Ice Storm.
- **Primal Fear**: Crime thriller with moral ambiguity, like Fargo or Cape Fear.

Now, compiling the top 10 from this list. The user's favorites include crime thrillers (Fargo, Usual Suspects), dramas with emotional depth (Jerry Maguire, Up Close and Personal), family stories (Postino, Big Night), and psychological elements (Game, Ice Storm). So movies like **Usual Suspects**, **Murder at 1600**, **Game**, **Big Night**, **That Thing You Do!**, **Courage Under Fire**, **Cape Fear**, **Chasing Amy**, **Sleepers**, and **Twelve Monkeys** seem to fit. Let me check if these are all in the Candidate Set.

Yes, all of123123123123 those are listed. Now, arranging them in order of relevance based on the themes. The answer should be 10 movies from the Candidate list that match the user's preferences.
</think>asdasdsadas

1. Usual Suspects  
2. Murder at 1600  
3. Game  
4. Big Night  
5. That Thing You Do!  
6. Courage Under Fire  
7. Cape Fear  
8. Chasing Amy  
9. Sleepers  
10. Twelve Monkeys"""

new = utils.clean_thinking(movie)
# remove tudo que está antes de 1.
#movie = re.sub(r"^.*?(\b[1-9]|10)\.", r"\1", movie)

print(new)

asdasdsadas

1. Usual Suspects  
2. Murder at 1600  
3. Game  
4. Big Night  
5. That Thing You Do!  
6. Courage Under Fire  
7. Cape Fear  
8. Chasing Amy  
9. Sleepers  
10. Twelve Monkeys


['ml_100k-gemma-3-12b-it-2025_05_13-15_46_59.pkl',
 'ml_100k-gemma-3-12b-it-2025_05_13-19_10_03.pkl',
 'ml_100k-gemma-3-12b-it-2025_05_14-16_51_47.pkl',
 'ml_100k-gemma-3-12b-it-2025_05_14-11_44_00.pkl',
 'ml_100k-gemma-3-12b-it-2025_05_12-15_13_02.pkl']

# geral

## testando limite de tokens

In [13]:
from Utils import utils 

model_name = "lmstudio-community/llama-3.2-3b-instruct"
system_prompt = "Você é um assistente que responde de forma simples."
temperatue = 0.0
max_tokens = -1
max_tokens_aux = 12000

prompt_base = "Testando token limite.\n"


step = 512

current_tokens = 6000
while current_tokens <= max_tokens_aux:
    prompt = prompt_base + " token" * current_tokens
    try:
        response = utils.query_lm_studio(model_name,temperatue,system_prompt,prompt,max_tokens)
        print(f"✅ Sucesso com {current_tokens} tokens")
        current_tokens += step
    except Exception as e:
        print(f"❌ Falhou em {current_tokens} tokens: {e}")
        break



✅ Sucesso com 6000 tokens
✅ Sucesso com 6512 tokens
✅ Sucesso com 7024 tokens
✅ Sucesso com 7536 tokens
✅ Sucesso com 8048 tokens
✅ Sucesso com 8560 tokens
✅ Sucesso com 9072 tokens
✅ Sucesso com 9584 tokens
✅ Sucesso com 10096 tokens
✅ Sucesso com 10608 tokens
✅ Sucesso com 11120 tokens
✅ Sucesso com 11632 tokens


In [2]:
import pickle
import pandas as pd

In [3]:

arq_name = 'Data/ML100K_train.pkl'

with open(f'{arq_name}', 'rb') as f:
    data = pickle.load(f)

In [2]:
data

{'config': {'LLM_runtime': 'ROCm llama.cpp v1.28.0',
  'dataset': 'ml_100k',
  'nsu': 12,
  'nci': 19,
  'lenlimit': 8,
  'lenlimit_option': 'aleatorio',
  'test_run': 0,
  'obs': 'WM config',
  'prompt_template': {'System_prompt': "You are a movie expert provide the answer for the question based on the given context. If you don't know the answer to a question, please don't share false information.",
   'Preference': '\n    ### MY RECENTLY WATCHED MOVIES (FROM OLDEST TO NEWEST): {}.\n\n    ### QUESTION: Based on my watched movies list. Tell me what features are most important to me when selecting movies (Summarize my preferences briefly)?\n\n    ### ANSWER:\n    ',
   'Featured_movies': '\n\n    ### MY RECENTLY WATCHED MOVIES (FROM OLDEST TO NEWEST): {}.\n\n    ### MY MOVIE PREFERENCES: {}.\n\n    ### QUESTION: Create an enumerated list selecting the five most featured movies from the watched movies according to my movie preferences.\n\n    ### ANSWER:\n    ',
   'Recommendation': '\n\

In [20]:
print(data['start_time'])

1743009974.9881866


In [ ]:
data['final_metrics']

{'hit': 0.07,
 'precision': 0.006999999999999999,
 'recall': 0.07,
 'ndcg': 0.027206193522998783}

In [ ]:
results = []
for key, value in data.items():
    if isinstance(key, int) and isinstance(value, dict):  # Pegando apenas os experimentos
        results.append({
            'Candidates': value.get('candidate_set', ''),
            'Ground Truth': value.get('ground_truth', ''),
            'gt_in_candidate_set': value.get('gt_in_candidate_set', ''),
            #'Input 1': value.get('input_1', ''),
            #'Predictions 1': value.get('predictions_1', ''),
            #'Input 2': value.get('input_2', ''),
            #'Predictions 2': value.get('predictions_2', ''),
            'Input 3': value.get('input_3', ''),
            'Predictions 3': value.get('predictions_3', ''),
            'Recommendations': value.get('recommendations', ''),
            'rec_HitRate@10': value.get('rec_HitRate@10', ''),
            'rec_NDCG@10': value.get('rec_NDCG@10', ''),
            #'Recall': value.get('recall', ''),
            #'NDCG': value.get('ndcg', '')
        })

df_results = pd.DataFrame(results)

df_results


,Candidates,Ground Truth,gt_in_candidate_set,Input 3,Predictions 3,Recommendations,rec_HitRate@10,rec_NDCG@10
0,"[Dave, Real Genius, Stand by Me, Die Hard: Wit...",Aristocats,no,"\n\n ### CANDIDATE MOVIE SET: Dave, Real Ge...",1. Batman \n2. Clear and Present Danger \n3. D...,,,
1,"[Star Trek: First Contact, Mission: Impossible...",River Wild,no,\n\n ### CANDIDATE MOVIE SET: Star Trek: Fi...,1. Usual Suspects \n2. Sleepers \n3. Chasing...,,,
2,"[Saint, Evita, Kull the Conqueror, Volcano, Fu...",Paradise Lost: The Child Murders at Robin Hood...,no,"\n\n ### CANDIDATE MOVIE SET: Saint, Evita,...",1. Apt Pupil \n2. Eve's Bayou \n3. In & Out ...,,,
3,"[Edge, Full Monty, Saint, Devil's Own, Money T...",Seven (Se7en),no,"\n\n ### CANDIDATE MOVIE SET: Edge, Full Mo...",1. Devil's Own\n2. Saint\n3. Ice Storm\n4. Kis...,,,
4,"[Pulp Fiction, Die Hard: With a Vengeance, Cha...",Beverly Hills Cop III,no,"\n\n ### CANDIDATE MOVIE SET: Pulp Fiction,...",1. Speed\n2. Die Hard: With a Vengeance\n3. Sc...,,,
...,...,...,...,...,...,...,...,...
95,"[Four Weddings and a Funeral, Blues Brothers, ...",Madonna: Truth or Dare,no,\n\n ### CANDIDATE MOVIE SET: Four Weddings...,1. Young Frankenstein \n2. Seven (Se7en) \n3...,,,
96,"[Birdcage, Indiana Jones and the Last Crusade,...",Crumb,no,"\n\n ### CANDIDATE MOVIE SET: Birdcage, Ind...",1. M*A*S*H (1970)\n2. Lawrence of Arabia (1962...,,,
97,"[Silence of the Lambs, One Flew Over the Cucko...",Birdcage,no,\n\n ### CANDIDATE MOVIE SET: Silence of th...,1. Young Frankenstein \n2. Chinatown \n3. Pu...,,,
98,"[Terminator, Terminator 2: Judgment Day, Die H...",My Best Friend's Wedding,no,"\n\n ### CANDIDATE MOVIE SET: Terminator, T...",1. Die Hard \n2. Aliens \n3. Speed \n4. Rum...,,,


In [ ]:
print(df_results)

                                            Candidates  \
0    [Speed (1994), True Lies (1994), Heathers (198...   
1    [Rock, The (1996), Four Weddings and a Funeral...   
2    [Money Talks (1997), Apt Pupil (1998), Rainmak...   
3    [Volcano (1997), L.A. Confidential (1997), Kis...   
4    [Raising Arizona (1987), Scream (1996), Shawsh...   
..                                                 ...   
395  [Multiplicity (1996), Twelve Monkeys (1995), L...   
396  [Clockwork Orange, A (1971), Fugitive, The (19...   
397  [Sound of Music, The (1965), Streetcar Named D...   
398  [Contact (1997), Executive Decision (1996), Go...   
399  [Money Talks (1997), Amistad (1997), Good Will...   

                                          Ground Truth gt_in_candidate_set  \
0                               Aristocats, The (1970)                  no   
1                               River Wild, The (1994)                 yes   
2    Paradise Lost: The Child Murders at Robin Hood...               

## Informações dataset

In [38]:
config = {
    "nsu": 12,  # Número de usuários similares
    "nci": 19,  # Número de itens candidatos
    "dataset": "ml_1m", # Nome do dataset
    }

In [39]:
from Utils import utils

user, gt_users = utils.total_users_and_gt_users(config)

print(f'Qnt de usuários do dataset: {user} \nQnt de usuários com gt no candidate: {gt_users}')

dataset = utils.read_json(f"Data/{config['dataset']}.json")

id_list = list(range(0, len(dataset)))

qtd_watched_mv = 0
avg_watched_mv = 0
highest_watched_mv = 0

for i in id_list:

    watched_mv = dataset[i][0].split(' | ')[::-1]
    qtd_watched_mv += len(watched_mv)
    if len(watched_mv) > highest_watched_mv:
        highest_watched_mv = len(watched_mv)

avg_watched_mv = qtd_watched_mv / len(id_list)
print(f'Quantidade de filmes assistidos: {qtd_watched_mv} \nMédia de filmes assistidos por usuário: {avg_watched_mv}\nMaior quantidade de filmes assistidos por usuário: {highest_watched_mv}')


Qnt de usuários do dataset: 6040 
Qnt de usuários com gt no candidate: 259
Quantidade de filmes assistidos: 994169 
Média de filmes assistidos por usuário: 164.5975165562914
Maior quantidade de filmes assistidos por usuário: 2313


In [12]:
## abrir df de .pkl
import pandas as pd
import pickle
def carregar_dataframe_pkl(caminho_arquivo):
    with open(caminho_arquivo, 'rb') as f:
        df = pickle.load(f)
    return df

df = carregar_dataframe_pkl('Data/ML100K_test.pkl')

In [27]:
df
## pega o valor do watched movies do primeiro usuário
#watched_movies = df.iloc[0]['watched_movies']
#watched_movies

#ground_truth = df.iloc[0]['ground_truth']
#ground_truth

,user_id,candidate_items,ground_truth,watched_movies
0,399,"[Murder at 1600, Desperate Measures, Amistad, ...",Dante's Peak,"[Scream, Titanic, Contact, Jackal, Full Monty,..."
1,237,"[Mars Attacks!, Eraser, Rock, Leaving Las Vega...",First Wives Club,"[Men in Black, Phenomenon, In & Out, Air Force..."
2,569,"[Volcano, Rosewood, Full Monty, Edge, Murder a...",Peacemaker,"[Seven Years in Tibet, Scream, Starship Troope..."
3,251,"[Ice Storm, Cold Comfort Farm, Scream, Mission...",Lone Star,"[Ridicule, Jane Eyre, Sense and Sensibility, T..."
4,389,"[Ulee's Gold, Leaving Las Vegas, Chasing Amy, ...",Mighty Aphrodite,"[Desperate Measures, Anna Karenina, Red Corner..."
5,613,"[Mr. Holland's Opus, Chasing Amy, L.A. Confide...",Nutty Professor,"[People vs. Larry Flynt, Postino, Il, Toy Stor..."
6,425,"[Return of the Jedi, Get Shorty, Apollo 13, Te...",Duck Soup,"[Paths of Glory, Father of the Bride, Quiet Ma..."
7,383,"[Seven Years in Tibet, Scream, G.I. Jane, Boog...",Evita,"[Alien: Resurrection, Contact, Titanic, Englis..."
8,650,"[In & Out, Air Force One, Scream, Big Night, S...",In & Out,"[Cold Comfort Farm, Secrets & Lies, Godfather,..."
9,890,"[Toy Story, Liar Liar, Courage Under Fire, Sta...",Courage Under Fire,"[Heaven's Prisoners, Broken Arrow, Rock, Godfa..."


In [15]:
# Criar a lista de listas
resultado = [[
    " | ".join(row['watched_movies']),
    row['ground_truth']
] for _, row in df.iterrows()]

print(resultado)

# Salvar a lista de listas em um arquivo json
import json
def salvar_json(caminho_arquivo, dados):
    with open(caminho_arquivo, 'w') as f:
        json.dump(dados, f, indent=4)

salvar_json('Data/ML100K_test_2.json', resultado)

[['Scream | Titanic | Contact | Jackal | Full Monty | Mrs. Brown (Her Majesty, Mrs. Brown) | English Patient | Air Force One', "Dante's Peak"], ['Men in Black | Phenomenon | In & Out | Air Force One | Liar Liar | Anastasia | Contact | English Patient', 'First Wives Club'], ["Seven Years in Tibet | Scream | Starship Troopers | Ice Storm | Ulee's Gold | Contact | Boogie Nights | L.A. Confidential", 'Peacemaker'], ['Ridicule | Jane Eyre | Sense and Sensibility | Twelve Monkeys | Ransom | Chasing Amy | English Patient | Air Force One', 'Lone Star'], ['Desperate Measures | Anna Karenina | Red Corner | Everyone Says I Love You | Fly Away Home | Contact | English Patient | L.A. Confidential', 'Mighty Aphrodite'], ['People vs. Larry Flynt | Postino, Il | Toy Story | Dead Man Walking | When We Were Kings | Broken Arrow | Evita | Scream', 'Nutty Professor'], ["Paths of Glory | Father of the Bride | Quiet Man | Singin' in the Rain | East of Eden | Kiss the Girls | Evita | Red Corner", 'Duck Soup'

## comparando resultados de diferentes parametros

In [19]:
import pickle
paths = [
    "Results/ml_100k-zs-nir-su18-ci19-llama-3.2-3b-instruct-2025_03_31-07_52_46.pkl",
    "Results/ml_100k-zs-nir-su18-ci19-llama-3.2-3b-instruct-2025_03_31-08_07_54.pkl",
    "Results/ml_100k-zs-nir-su18-ci25-llama-3.2-3b-instruct-2025_03_31-08_44_11.pkl",
    "Results/ml_100k-zs-nir-su20-ci30-llama-3.2-3b-instruct-2025_03_31-09_09_17.pkl"
]

data = []

for path in paths:
    with open(path, "rb") as f:
        data.append(pickle.load(f))

# Agora data contém os resultados carregados


In [20]:
data[0]['metrics']['hit']
data[0]['metrics']['ndcg']
data[0]['metrics']['precision']
data[0]['metrics']['recall']
data[0]['runtime']
data[0]['config']['nci']
data[0]['config']['nsu']
data[0]['config']['lenlimit']

8

In [21]:
import pandas as pd

# Criar lista para armazenar os dados estruturados
table_data = []

# Iterar sobre os resultados carregados
for i, result in enumerate(data):  # Itera sobre a lista
    table_data.append({
        "Teste": paths[i],  # Usa o path como identificador
        "NCI": result["config"]["nci"],
        "NSU": result["config"]["nsu"],
        "LenLimit": result["config"]["lenlimit"],
        "Hit": result["metrics"]["hit"],
        "NDCG": result["metrics"]["ndcg"],
        "Precision": result["metrics"]["precision"],
        "Recall": result["metrics"]["recall"],
        "Runtime": result["runtime"]
    })

# Criar DataFrame
df = pd.DataFrame(table_data)

df

,Teste,NCI,NSU,LenLimit,Hit,NDCG,Precision,Recall,Runtime
0,Results/ml_100k-zs-nir-su18-ci19-llama-3.2-3b-...,19,18,8,0.07,0.027206,0.007000,0.07,585.857143
1,Results/ml_100k-zs-nir-su18-ci19-llama-3.2-3b-...,19,18,16,0.07,0.025125,0.006833,0.07,779.467403
2,Results/ml_100k-zs-nir-su18-ci25-llama-3.2-3b-...,25,18,25,0.09,0.038035,0.009000,0.09,881.761403
3,Results/ml_100k-zs-nir-su20-ci30-llama-3.2-3b-...,30,20,30,0.05,0.029889,0.005000,0.05,944.329995
